In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Pancreatic_Cancer"
cohort = "GSE183795"

# Input paths
in_trait_dir = "../../input/GEO/Pancreatic_Cancer"
in_cohort_dir = "../../input/GEO/Pancreatic_Cancer/GSE183795"

# Output paths
out_data_file = "../../output/preprocess/Pancreatic_Cancer/GSE183795.csv"
out_gene_data_file = "../../output/preprocess/Pancreatic_Cancer/gene_data/GSE183795.csv"
out_clinical_data_file = "../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE183795.csv"
json_path = "../../output/preprocess/Pancreatic_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Microarray gene-expression profiles of 139 pancreatic tumor,102 adjacent non-tumor tissue  from patients with pancreatic ductal adenocarcinoma  and 3 normal pancreas from donors."
!Series_summary	"In order to identify key pathways associated with disease aggressiveness and therapeutics resistance in the most agrressive subset of PDAC, we analyzed gene expression profiling of tumor and adjacent non-tumor tissues from PDAC cases. Non-negative matrix factorization (NMF) clustering, using gene expression profile form PDAC tumors, revealed three patient subsets. A 142-gene signature specific to the subset with the worst patient survival, predicted prognosis and stratified patients with significantly different survival. Mechanistic and functional analyses of most aggressive subset revealed a HNF1B/Clusterin Axis negatively regulate pancreatic cancer progression and potentially be useful in designing novel strategies to attenuate disease progression. Aff

### Step 2: Data Normalization and Linking

In [3]:
# First, we need to extract gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_data.shape}")

# Check what kind of identifiers are in the gene expression data
print(f"Example gene identifiers: {list(gene_data.index[:5])}")

# Extract gene annotation from the SOFT file to map probe IDs to gene symbols
gene_annotation = get_gene_annotation(soft_file)
print(f"Gene annotation shape: {gene_annotation.shape}")

# Before mapping, let's see what columns are available in the annotation data
print(f"Gene annotation columns: {gene_annotation.columns.tolist()}")

# Get the mapping between probe IDs and gene symbols
# Assuming 'ID' is the probe column and 'GENE_SYMBOL' or similar is the gene symbol column
# Let's identify the gene symbol column
gene_symbol_cols = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower()]
if gene_symbol_cols:
    gene_col = gene_symbol_cols[0]
    print(f"Using gene symbol column: {gene_col}")
    gene_mapping = get_gene_mapping(gene_annotation, "ID", gene_col)
    
    # Apply gene mapping to convert probe IDs to gene symbols
    gene_data_mapped = apply_gene_mapping(gene_data, gene_mapping)
    print(f"Mapped gene data shape: {gene_data_mapped.shape}")
    
    # 1. Now normalize gene symbols in the gene expression data
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data_mapped)
    print(f"Normalized gene data shape: {normalized_gene_data.shape}")
else:
    # If we can't find a gene symbol column, try using the data as is
    print("Could not identify gene symbol column, using probe IDs directly")
    normalized_gene_data = gene_data
    print(f"Using original gene data shape: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features based on the sample characteristics we observed
def convert_tissue_type(value):
    """Convert tissue type to binary values (0: non-tumor, 1: tumor)"""
    if pd.isna(value) or not isinstance(value, str):
        return None
    
    value = value.lower()
    if "tumor" in value and "non-tumor" not in value:
        return 1  # Tumor
    elif "non-tumor" in value or "normal" in value:
        return 0  # Non-tumor or normal
    else:
        return None

# Extract clinical features - tissue type is our primary trait for pancreatic cancer
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait="Pancreatic_Cancer",
    trait_row=0,  # Tissue type is in position 0
    convert_trait=convert_tissue_type,
    age_row=None,  # Age doesn't appear to be in the dataset
    convert_age=None,
    gender_row=None,  # Gender doesn't appear to be in the dataset
    convert_gender=None
)

# Save clinical features
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, "Pancreatic_Cancer")
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, "Pancreatic_Cancer")

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Cohort contains pancreatic cancer tumor and non-tumor tissue samples."
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data quality check failed. Linked data not saved.")

Gene expression data shape: (19245, 244)
Example gene identifiers: ['7896748', '7896754', '7896756', '7896761', '7896798']


Gene annotation shape: (4729321, 12)
Gene annotation columns: ['ID', 'GB_LIST', 'SPOT_ID', 'seqname', 'RANGE_GB', 'RANGE_STRAND', 'RANGE_START', 'RANGE_STOP', 'total_probes', 'gene_assignment', 'mrna_assignment', 'category']
Using gene symbol column: gene_assignment


Mapped gene data shape: (100969, 244)
Normalized gene data shape: (20022, 244)


Normalized gene data saved to ../../output/preprocess/Pancreatic_Cancer/gene_data/GSE183795.csv
Clinical features saved to ../../output/preprocess/Pancreatic_Cancer/clinical_data/GSE183795.csv
Linked data shape before handling missing values: (244, 20023)


Linked data shape after handling missing values: (244, 20023)
For the feature 'Pancreatic_Cancer', the least common label is '0.0' with 105 occurrences. This represents 43.03% of the dataset.
The distribution of the feature 'Pancreatic_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Pancreatic_Cancer/GSE183795.csv
